In [ ]:
import numpy as np
import cupy as cp
print("cupy: ", cp.__version__)

In [ ]:
def extract_coincidentals(spikes_list, idx):
    # Spikes coordinates at given wavelength index
    spikes_w = spikes_list[idx]
    # Associated neighbour coordinates
    nb_pixels = index_8nb[spikes_w[0, :], :]
    # Sublist of spikes data that will excludes the one serving as template
    spikes_sublist = spikes_list[:idx] + spikes_list[idx + 1:]
    # Coincidental cross-referencing.
    # mask_w_arr = np.array([np.isin(nb_pixels, index_8nb[spikes[0, :], :]).any(axis=1) for spikes in spikes_sublist])
    mask_w_arr = np.array([np.isin(nb_pixels, spikes[0, :]).any(axis=1) for spikes in spikes_sublist])
    select_pixels = mask_w_arr.any(axis=0)
    coords_w = spikes_w[0, select_pixels]
    w_tables = np.insert(mask_w_arr[:, select_pixels], idx, True, axis=0)
    # Retrieve intensity values for the selected coordinates
    intensities = spikes_w[1:, select_pixels]
    arr_w = np.concatenate([coords_w[np.newaxis, ...], intensities, w_tables], axis=0)
    arr_w = np.insert(arr_w, 3, idx, axis=0)

    return arr_w


def process_group(group_n):
    fpaths = path_Series.loc[group_n]
    spikes_list = [fitsio.read(os.path.join(os.environ['SPIKESDATA'], f)) for f in fpaths]
    group_data = np.concatenate([extract_coincidentals(spikes_list, i) for i in range(7)], axis=1)
    column_names = ['coords', 'int1', 'int2', 'wref', 'w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6']
    coincidental_spikes_df = pd.DataFrame(group_data.T, columns=column_names)
    coincidental_spikes_df['GroupNumber'] = group_n
    return coincidental_spikes_df


def xy_coords(coords_1d):
    x = coords_1d % 4096
    y = coords_1d // 4096
    return x, y


@cp.fuse()
def dist_mat(x1, x2, y1, y2):
    return (x1 - x2)*(x1 - x2) + (y1-y2)*(y1-y2)


def dist_matrix(coords1, coords2):
    # Calculate distance matrix between 2 sets of coordinates. 
    # Inputs are 2 lists of spikes 1D-ravelled coordinates 
    
    # unravel to the 2D coordinate system of the CCD (4096 x 4096). This is faster than unravel() functions
    x1, y1 = xy_coords(coords1)
    x2, y2 = xy_coords(coords2)
    # Use broadcasting to get Euclidian distances. Seemed faster than using meshgrid functions.
    x1b = x1[:, np.newaxis]
    y1b = y1[:, np.newaxis]

    dist_matrix = dist_mat(x2, x1b, y2, y1b)#(x2 - x1b)**2 + (y2 - y1b)**2
    
    return dist_matrix


def is_near(coords1, coords2, distance):
    # Get a boolean array of same size as coords1 assigning True / False to its elements within 'distance' from coords2
    dmat = dist_matrix(coords1, coords2)
    near_mask = dmat <= distance
    is_near = near_mask.any(axis=1)
    return is_near

def is_near_w(coords_, distance, widx):
    # Get the pixels to keep within one file at a given wavelength
    isnear_bools = [is_near(coords_[widx][0:8000], coords_[i][0:8000], 2) for i in range(7)]
    isnear_bools[widx] = cupy.triu(isnear_bools[widx], k=1)
    mask_w_arr = cupy.vstack(isnear_bools)
    return mask_w_arr

def where_coincidentals_cupy(spikes_list, widx):
    
    cucoords_ = [cupy.asarray(spikes) for spikes in spikes_list]
    mask_w_arr = is_near_w(cucoords_, 2, widx)
    select_pixels = mask_w_arr.any(axis=0)
    return select_pixels.get()


    
# def process_group(fpaths):
    
#     spikes_list = [fitsio.read(os.path.join(os.environ['SPIKESDATA'], f)) for f in fpaths]
#     cucoords_ = [cupy.asarray(spikes[0,:]) for spikes in spikes_list]
#     isnear_w = [is_near(cucoords_[0][0:8000], cucoords_[i][0:8000], 2) for i in range(7)]
#     # 1st array is the "self" same-wavelength distance matrix => Must reject the redundancies 
#     # It is by definition always symmetric. Elimitate redundancies 
#     # by zeroing the lower triangle, including the diagonal (hence k=1). 
#     isnear_w[0] = cupy.triu(isnear_[0], k=1)
#     bool_pixels = cupy.vstack(isnear_w).any(axis=0)
   
#     return isnear0

In [ ]:
nspikes = [8486, 30_356, 36_549, 8200, 13_781, 26_443, 27_576]
spikes_list = [np.random.randint(1, high=(4096*4096)-1, size=n, dtype=np.int32) for n in nspikes]

## Load them in cpu RAM

In [ ]:
pixs = where_coincidentals_cupy(spikes_list, 0)

In [ ]:
%timeit a, b = extract_coincidentals_w_cupy(spikes_list, 0)